###Python setup

In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


In [4]:
from fastbook import *
from fastai.text.all import *
import pandas as pd

###Load abstract data

In [5]:
### Load data from github
url = 'https://github.com/andreasebbehoj/EASD-abstract-nlp/raw/main/2_Data/Abstracts.csv'
df = pd.read_csv(url, delimiter=';')
df.head(n=1)

,id,publicationyear,page,oral,title,abstract,authornames,authoraddresses,correspondenceaddress,emtreedrugindextermsmajorf,emtreedrugindexterms,emtreemedicalindextermsmajo,emtreemedicalindexterms,casregistrynumbers,embaselink
0,1,2018,3,Oral,Methylglyoxal induces microglial polarisation to pro-inflammatory state,"BACKGROUND AND AIMS: Diabetic retinopathy induces vasoregression and neurodegeneration and finally leads to blindness. Microglial activation is involved in diabetic retinopathy. Recently our study showed that exogenous AGE precursor methylglyoxal activates and recruits microglia in normoglycemic retina. Different functional properties of microglia are dependent on the state of polarization. Classical activation of M1 phenotype is pro-inflammation and alternative activation of M2 phenotype is anti-inflammation. However, the role of methylglyoxal in microglial polarization remains unclear. H...","Lin J., Wang J., Schlotterer A., Kolibabka M., Dietrich N., Fleming T., Hammes H.","(Lin J., Wang J., Schlotterer A., Kolibabka M., Dietrich N., Hammes H.) Medical Faculty Mannheim, Mannheim, Germany. , (Fleming T.) Medical Faculty Heidelberg, Heidelberg, Germany.","J. Lin, Medical Faculty Mannheim, Mannheim, Germany.",methylglyoxal,"allograft inflammatory factor 1, arginase 1, CD74 antigen, endogenous compound, galectin 3, intercellular adhesion molecule 1, interleukin 1, macrophage inflammatory protein 1alpha, monocyte chemotactic protein 1, tumor necrosis factor",polarization,"animal cell, animal experiment, C57BL 6 mouse, conference abstract, controlled study, genetic marker, immunofluorescence, in vitro study, in vivo study, inflammation, microglia, mouse, nonhuman, phenotype, precursor, protein expression, retina","allograft inflammatory factor 1 (200890-56-8), galectin 3 (208128-56-7), intercellular adhesion molecule 1 (126547-89-5), macrophage inflammatory protein 1alpha (155075-84-6), methylglyoxal (78-98-8)",https://www.embase.com/search/results?subaction=viewrecord&id=L624031056&from=export


In [6]:
### Load data into language model datablock
dls_lm = DataBlock(
    blocks=TextBlock.from_df('abstract', is_lm=True), ## abstract is the var in df that contains text
    get_x=ColReader('text'), ## should always be 'text' irrespective of var name (according to FastAI tutorial)
    splitter=RandomSplitter(0.2, seed=42) ## Split into train/validation data
).dataloaders(df, bs=128, seq_len=80)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [7]:
### Show batch after automatic text-preprocessing  
dls_lm.show_batch(max_n=3)

,text,text_
0,"xxbos xxup background xxup and xxup aims : xxmaj high glucose - induced damage in microvascular cells in vitro and progression of retinopathy and nephropathy in diabetic animals are prevented by thiamine supplementation . xxmaj impaired thiamine availability facilitates metabolic damage , and renal loss of this vitamin is described in diabetic patients . xxmaj two snps located in the xxup xxunk gene encoding for the thiamine transporter-2 ( thtr-2 ) are associated with resistance to development of proliferative diabetic","xxup background xxup and xxup aims : xxmaj high glucose - induced damage in microvascular cells in vitro and progression of retinopathy and nephropathy in diabetic animals are prevented by thiamine supplementation . xxmaj impaired thiamine availability facilitates metabolic damage , and renal loss of this vitamin is described in diabetic patients . xxmaj two snps located in the xxup xxunk gene encoding for the thiamine transporter-2 ( thtr-2 ) are associated with resistance to development of proliferative diabetic retinopathy"
1,". xxmaj they also suggest that a negative xxup cvd history could be used as a criterion to select those xxup t2d patients who would most benefit from intensive glycemic control for xxup can prevention , whereas xxup bp control is effective regardless of xxup cvd history . xxbos xxup background xxup and aims : we assessed the safety and effectiveness of xxunk night hybrid closed - loop insulin delivery compared with sensor augmented pump therapy in youths and adults","xxmaj they also suggest that a negative xxup cvd history could be used as a criterion to select those xxup t2d patients who would most benefit from intensive glycemic control for xxup can prevention , whereas xxup bp control is effective regardless of xxup cvd history . xxbos xxup background xxup and aims : we assessed the safety and effectiveness of xxunk night hybrid closed - loop insulin delivery compared with sensor augmented pump therapy in youths and adults with"
2,"and cancer mortality . xxmaj here , we test the hypotheses that : ( i ) most cancer deaths are from obesity - related cancers ( orcs ) , and ( ii ) among individuals with xxup t2d , there is a positive association between xxup bmi ( at diagnosis of xxup t2d ) and cancer mortality . xxup materials xxup and xxup methods : xxmaj using the xxup uk xxmaj clinical xxmaj practice xxmaj research xxmaj datalink , an","cancer mortality . xxmaj here , we test the hypotheses that : ( i ) most cancer deaths are from obesity - related cancers ( orcs ) , and ( ii ) among individuals with xxup t2d , there is a positive association between xxup bmi ( at diagnosis of xxup t2d ) and cancer mortality . xxup materials xxup and xxup methods : xxmaj using the xxup uk xxmaj clinical xxmaj practice xxmaj research xxmaj datalink , an incident"


##Train language model
First part loads the pre-trained model (based on English wiki text). Then the model gets fine-tuned on the EASD abstracts. 

This model's final layer can predict the next word in a sentence. This can be used for auto-generating (poor) abstracts. But the model can also be trained for classifying abstracts in those accepted for an oral presentation vs only a poster presentation.

In [8]:
### Load pretrained model
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [9]:
# Fine-tune last layer on our data
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.933551,3.709484,0.351179,40.832718,00:28


In [10]:
# Unfreeze and continued fine-tuning of all layers
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.356617,3.606467,0.363622,36.835674,00:31
1,3.240782,3.530679,0.369375,34.147163,00:31
2,3.038583,3.472236,0.379998,32.208679,00:31
3,2.785288,3.471196,0.378058,32.175190,00:31
4,2.539963,3.502131,0.381725,33.186104,00:31
5,2.307567,3.527531,0.382124,34.039818,00:31
6,2.089313,3.594287,0.381518,36.389759,00:31
7,1.919999,3.654446,0.379215,38.646111,00:31
8,1.805562,3.687399,0.378123,39.940823,00:31
9,1.736844,3.703029,0.377165,40.570023,00:31


In [15]:
### Save model
# All layers
learn.save('lm_complete')

# All layers except the last (for future classification model)
learn.save_encoder('lm_encoder')

### Use language model to auto-generate abstracts
Note: they are fun but absolutely awful. Don't submit them anywhere. 

In [12]:
### Abstract-generator
TEXTSTART = ["Chronic kidney disease", "Metformin is", "Sodium glucose cotransporter-2 inhibitors"]
N_WORDS = 400
N_SENTENCES = 3
Abstracts = []

for n in range(N_SENTENCES):
  preds = learn.predict(TEXTSTART[n], N_WORDS, temperature=0.75)
  Abstracts.append(preds)

print("\n".join(Abstracts))

Chronic kidney disease ( ckd ) CKD is a non - invasive , randomised ( 1:1 ) trial designed to reduce the risk of cardiovascular ( cv ) outcomes and reduce the risk of CV death by 38 % vs placebo once an estimated insulin release event rate ( range ) of coronary artery disease or stroke . The aim of the present study was to evaluate the effect of EMPA on CV death , stroke , and all - cause mortality in patients with type 2 diabetes mellitus ( t2 dm ) in the LEADER trial . MATERIALS AND METHODS : This was a 24-week multicentre , multicentre , randomised , parallel - group , placebo - controlled trial . Patients in the LEADER trial were randomized to receive once - weekly placebo , empagliflozin 10 mg , or liraglutide 1.8 mg . Participants were randomized 1:1 to switch 1 mg / day or placebo over 5 weeks . Liraglutide significantly reduced hba1c and body weight by baseline subgroup ( p < 0.001 ) . Changes in hba1c and body weight were also assessed . The primary outcome was time to resolut